### Model Learning an Image with just Normal Multi Layer perceptron (Dense Neural Network)

In [2]:
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense,  Flatten

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import cv2
import os
from tensorflow.keras.callbacks import Callback

In [9]:

image_path = "image.jpg" 
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 
image_resized = cv2.resize(image, (150, 150))  # adjust you image size and then change everywhere accordingly
image_normalized = image_resized / 255.0  # Normalize pixel values to [0, 1]

# # Expand dimensions to make it compatible with InceptionV3 (convert grayscale to RGB)
# image_rgb = np.stack([image_normalized] * 3, axis=-1)  # Convert grayscale to 3 channels
# image_rgb = np.expand_dims(image_rgb, axis=0) 

image_bw = np.expand_dims(image_normalized, axis=0)  

model = Sequential([
    # Flatten the input
    Flatten(input_shape=(150, 150, 1)), 
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(150*150, activation='sigmoid'),
    
])
model.summary() 

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 22500)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │       360,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 22500)          │       382,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 742,788 (2.83 MB)

 Trainable params: 742,788 (2.83 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:

class SaveReconstructedImageCallback(Callback):
    def __init__(self, model_input, original_shape, output_dir="normal_learned_images", background_color='black'):
        super().__init__()
        self.model_input = model_input
        self.original_shape = original_shape
        self.output_dir = output_dir
        self.background_color = background_color
        os.makedirs(self.output_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        # Predict the reconstructed image
        reconstructed = self.model.predict(self.model_input)
        reconstructed_image = reconstructed.reshape(self.original_shape)

        # Save the reconstructed image
        reconstructed_image = np.clip(reconstructed_image, 0, 1)

        img_name = (f"0{epoch}" if epoch<10 else f"{epoch}")
        save_path = os.path.join(self.output_dir, f'epoch_normal{img_name}.png')

        # Create a figure with specified background color
        fig, ax = plt.subplots(facecolor=self.background_color) 

        # Display the image and remove axis
        ax.imshow(reconstructed_image, cmap='gray')
        # ax.axis('off') 

        # Add title
        title = f"Normal NN, Epoch {epoch+1}, Loss: {logs['loss']:.4f}"
        ax.set_title(title)

        # Save the figure
        plt.savefig(save_path, bbox_inches='tight') 
        plt.close()

        print(f"Normal NN Learned image saved at epoch {epoch + 1}")

In [11]:

X_train = image_bw   #use image_rgb for 3 channel image
Y_train = image_normalized.flatten()  
original_shape = (150, 150)  

save_callback = SaveReconstructedImageCallback(model_input=X_train, original_shape=original_shape)


model.compile(optimizer='adam', loss='mse')
model.fit(X_train, np.expand_dims(Y_train, axis=0), epochs=40, batch_size=1, callbacks=[save_callback])

print("Training complete. Reconstructed images saved in:", save_callback.output_dir)

Epoch 1/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.071
Normal NN Learned image saved at epoch 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.0717
Epoch 2/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.070
Normal NN Learned image saved at epoch 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - loss: 0.0709
Epoch 3/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.066
Normal NN Learned image saved at epoch 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - loss: 0.0668
Epoch 4/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.058
Normal NN Learned image saved at epoch 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - loss: 0.0588
Epoch 5/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.048
Normal NN Learned image saved at epoch 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - loss: 0.0481
Epoch 6/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.036
Normal NN Learned image saved at epoch 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step - loss: 0.0367
Epoch 7/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 

In [12]:
# Create a video from the saved images
output_dir="normal_learned_images"

def create_video_from_images(image_dir, video_name, fps=2):
    images = sorted([img for img in os.listdir(image_dir)])
    frame = cv2.imread(os.path.join(image_dir, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_dir, image)))

    cv2.destroyAllWindows()
    video.release()

# Generate video
video_name = f"learning_normal_image.mp4"
create_video_from_images(output_dir, video_name, fps=2)

print(f"Video saved as '{video_name}'")


Video saved as 'learning_normal_image.mp4'
